## Wallaroo Dashboard Metrics Retrieval Tutorial

The following tutorial demonstrates using the Wallaroo MLOps API to retrieve Wallaroo metrics data.  These requests are compliant with Prometheus API endpoints.  

This tutorial lists the metrics queries available and demonstrates how to perform each of the queries.

### Prerequisites

This tutorial assumes the following:

* A Wallaroo Ops environment is installed.
* The Wallaroo SDK is installed.  These examples use the Wallaroo SDK to generate the initial inferences information for the metrics requests.

## Inference Data Generation

This part of the tutorial generates the inference results used for the rest of the tutorial.

### Import libraries

The first step is to import the libraries required.

In [1]:
import json
import numpy as np
import pandas as pd

import pytz
import datetime

import requests
from requests.auth import HTTPBasicAuth

import wallaroo

### Connect to the Wallaroo Instance

A connection to Wallaroo is established via the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client(api_endpoint="https://autoscale-uat-gcp.wallaroo.dev/", 
                     auth_type="sso")



Please log into the following URL in a web browser:

	https://autoscale-uat-gcp.wallaroo.dev/auth/realms/master/device?user_code=RQRE-DPPU

Login successful!


In [3]:
model_name = "ccfraud-model"
model_file_name = "./models/ccfraud.onnx"


The following queries are available for resource consumption.  Note where each request either uses the `query` endpoint or the `query_range` endpoint.

| Query Name | API Route | Example Query | Description | 
|---|---|---|---|
| Total CPU Requested | query | `sum(wallaroo_kube_pod_resource_requests{resource="cpu"})` | Number of CPUs requested in the Wallaroo cluster |
| Total CPU allocated | query | `sum(kube_node_status_capacity{resource="cpu"})` | Total number of available CPUs in the Wallaroo cluster |
| Total GPU Requested | query | `sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})` | Number of GPUs requested in the Wallaroo cluster |
| Total GPU Allocated | query | `sum(kube_node_status_capacity{resource=~"nvidia_com_gpu\|qualcomm_com_qaic"})` | Total number of available GPUs in the Wallaroo cluster |
| Total Memory Requested | query | `sum(wallaroo_kube_pod_resource_requests{resource="memory"})` | Amount of memory requested in the Wallaroo cluster. |
| Total Memory Allocated | query | `sum(kube_node_status_capacity{resource="memory"})` | Total amount of memory available in the Wallaroo cluster. |
| Total Inference Log Storage used | query | `kubelet_volume_stats_used_bytes{persistentvolumeclaim="plateau-managed-disk"}` | Amount of inference log storage used. |
| Total Inference Log Storage allocated | query | `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="plateau-managed-disk"}` | Total amount of inference log storage available. |
| Total Artifact Storage used | query | `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="minio"}` | Amount of model and orchestration artifact storage used. |
| Total Artifact Storage allocated | query | `kubelet_volume_stats_used_bytes{persistentvolumeclaim="minio"}` | Total amount of model and orchestration artifact storage available. |
| Average GPU usage over time | query_range | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[1h:] offset 1h)` | Average GPU usage over the defined time range in the Wallaroo cluster. |
| Average GPU requested over time | query_range | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[1h:] offset 1h)` | Average number of GPU requested over the defined time range in the Wallaroo cluster | 
| Average CPU usage over time | query_range | `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="cpu"})[1h:] offset 1h)` | Average CPU usage over the defined time range in the Wallaroo cluster. |
|  Average CPU requested over time | query_range | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="cpu"})[1h:] offset 1h)` | Average CPU requests over the defined time range in the Wallaroo cluster |
| Average Memory usage over time | query_range | `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="memory"})[1h:] offset 1h)` | Average memory usage over the defined time range in the Wallaroo cluster. |
| Average Memory requests over time | query_range | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="memory"})[1h:] offset 1h)` | Average memory requests over the defined time range in the Wallaroo cluster. |
| Average pipelines CPU usage over time | query_range | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_usage{resource="cpu"})[1h:] offset 1h)` | Average CPU usage over the defined time range for an individual Wallaroo pipeline.
| Average pipelines CPU requested over time | query_range | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource="cpu"})[1h:] offset 1h)` | Average number of CPUs requested over the defined time range for an individual Wallaroo pipeline. |
| Average pipelines GPU usage over time | query_range | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[1h:] offset 1h)` | Average GPU usage over the defined time range for an individual Wallaroo pipeline. |
| Average pipelines GPU requested over time | query_range | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[1h:] offset 1h)` | Average number of GPUs requested over the defined time range for an individual Wallaroo pipeline. |
| Average pipelines Mem usage over time | query_range | `avg_over_time(sum by(namespace) (wallaroo_kube_pod_resource_usage{resource="memory"})[1h:] offset 1h)` | Average memory usage over the defined time range for an individual Wallaroo pipeline. |
| Average pipelines Mem requested over time | query_range | `avg_over_time(sum by (namespace)(wallaroo_kube_pod_resource_requests{resource="memory"})[1h:] offset 1h)` | Average amount of memory requested over the defined time range for an individual Wallaroo pipeline. |
| Pipeline inference log storage | query_range | `avg_over_time(sum by(topic) (topic_bytes)[1h:] offset 1h)` | Average inference log storage used over the defined time range for an individual Wallaroo pipeline |

### Total CPU Requested

* Total CPU Requested
* query 
* `sum(wallaroo_kube_pod_resource_requests{resource="cpu"})`
* Number of CPUs requested in the Wallaroo cluster


In [19]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'sum(wallaroo_kube_pod_resource_requests{resource="cpu"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764100020.302, '9.306000000000001']}]}}

### Total CPU allocated

* Total CPU allocated
* query
* `sum(kube_node_status_capacity{resource="cpu"})`
* Total number of available CPUs in the Wallaroo cluster


In [20]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'sum(kube_node_status_capacity{resource="cpu"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764100024.924, '48']}]}}

### Total GPU Requested

* Total GPU Requested
* query
* `sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})`
* Number of GPUs requested in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764100195.103, '2']}]}}

### Total GPU Allocated

* Total GPU Allocated
* query
* `sum(kube_node_status_capacity{resource=~"nvidia_com_gpu|qualcomm_com_qaic"})` 
* Total number of available GPUs in the Wallaroo cluster


In [22]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'sum(kube_node_status_capacity{resource=~"nvidia_com_gpu|qualcomm_com_qaic"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764100578.417, '5']}]}}

### Total Memory Requested

* Total Memory Requested
* query
* `sum(wallaroo_kube_pod_resource_requests{resource="memory"})`
* Amount of memory requested in the Wallaroo cluster.


In [23]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'sum(wallaroo_kube_pod_resource_requests{resource="memory"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764100633.198, '27791458304']}]}}

### Total Memory Allocated

* Total Memory Allocated
* query
* `sum(kube_node_status_capacity{resource="memory"})`
* Total amount of memory available in the Wallaroo cluster.


In [24]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'sum(kube_node_status_capacity{resource="memory"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764100709.531, '197850009600']}]}}

### Total Inference Log Storage used

* Total Inference Log Storage used
* query
* `kubelet_volume_stats_used_bytes{persistentvolumeclaim="plateau-managed-disk"}`
* Amount of inference log storage used.



In [25]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'kubelet_volume_stats_used_bytes{persistentvolumeclaim="plateau-managed-disk"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_used_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kubernete

### Total Inference Log Storage allocated

* Total Inference Log Storage allocated
* query
* `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="plateau-managed-disk"}`
* Total amount of inference log storage available.



In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="plateau-managed-disk"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_capacity_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kuber

### Total Artifact Storage used

* Total Artifact Storage used
* query
* `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="minio"}`
* Amount of model and orchestration artifact storage used.



In [27]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="minio"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_capacity_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kuber

Total Artifact Storage allocated

* Total Artifact Storage allocated
* query
* `kubelet_volume_stats_used_bytes{persistentvolumeclaim="minio"}`
* Total amount of model and orchestration artifact storage available.



In [28]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'kubelet_volume_stats_used_bytes{persistentvolumeclaim="minio"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_used_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kubernete

### Average GPU usage over time

* Average GPU usage over time
* query_range
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)`
* Average GPU usage over the defined time range in the Wallaroo cluster.


In [29]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764101141.07, '2']}]}}

### Average GPU requested over time

* Average GPU requested over time
* query_range
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)`
* Average number of GPU requested over the defined time range in the Wallaroo cluster




In [36]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {},
    'values': [[1764014400, '1'],
     [1764014700, '1'],
     [1764015000, '1'],
     [1764015300, '1'],
     [1764015600, '1'],
     [1764015900, '1'],
     [1764016200, '1'],
     [1764016500, '1'],
     [1764016800, '1'],
     [1764017100, '1'],
     [1764017400, '1'],
     [1764017700, '1'],
     [1764018000, '1'],
     [1764018300, '1'],
     [1764018600, '1'],
     [1764018900, '1'],
     [1764019200, '1'],
     [1764019500, '1'],
     [1764019800, '1'],
     [1764020100, '1'],
     [1764020400, '1'],
     [1764020700, '1'],
     [1764021000, '1'],
     [1764021300, '1'],
     [1764021600, '1'],
     [1764021900, '1'],
     [1764022200, '1'],
     [1764022500, '1'],
     [1764022800, '1'],
     [1764023100, '1'],
     [1764023400, '1'],
     [1764023700, '1'],
     [1764024000, '1'],
     [1764024300, '1'],
     [1764024600, '1'],
     [1764024900, '1'],
     [1764025200, '1'],
     [1764025500, 

### Average CPU usage over time

* Average CPU usage over time
* query_range
* `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="cpu"})[1h:] offset 1h)`
* Average CPU usage over the defined time range in the Wallaroo cluster.



In [38]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="cpu"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {},
    'values': [[1764014400, '0.20781380863333335'],
     [1764014700, '0.2030914302875'],
     [1764015000, '0.20102717032916667'],
     [1764015300, '0.19932740979722222'],
     [1764015600, '0.19710094489305555'],
     [1764015900, '0.19683597161388888'],
     [1764016200, '0.19538259814722223'],
     [1764016500, '0.19169714007083336'],
     [1764016800, '0.18914183558333333'],
     [1764017100, '0.18531675833333333'],
     [1764017400, '0.18414001672083335'],
     [1764017700, '0.18405097903333334'],
     [1764018000, '0.1833151287375'],
     [1764018300, '0.18240700316666666'],
     [1764018600, '0.17967357115416668'],
     [1764018900, '0.17665639894027776'],
     [1764019200, '0.17319259899444445'],
     [1764019500, '0.17079004260694447'],
     [1764019800, '0.17117557496527777'],
     [1764020100, '0.17250379108333336'],
     [1764020400, '0.1733866390625'],
     [1764020700, '0.173896527241666

### Average CPU requested over time

* Average CPU requested over time
* query_range
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="cpu"})[1h:] offset 1h)`
* Average CPU requests over the defined time range in the Wallaroo cluster





In [39]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="cpu"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {},
    'values': [[1764014400, '7.806'],
     [1764014700, '7.806'],
     [1764015000, '7.806'],
     [1764015300, '7.806'],
     [1764015600, '7.806'],
     [1764015900, '7.806'],
     [1764016200, '7.806'],
     [1764016500, '7.806'],
     [1764016800, '7.806'],
     [1764017100, '7.806'],
     [1764017400, '7.806'],
     [1764017700, '7.806'],
     [1764018000, '7.806'],
     [1764018300, '7.806'],
     [1764018600, '7.806'],
     [1764018900, '7.806'],
     [1764019200, '7.806'],
     [1764019500, '7.806'],
     [1764019800, '7.806'],
     [1764020100, '7.806'],
     [1764020400, '7.806'],
     [1764020700, '7.806'],
     [1764021000, '7.806'],
     [1764021300, '7.806'],
     [1764021600, '7.806'],
     [1764021900, '7.806'],
     [1764022200, '7.806'],
     [1764022500, '7.806'],
     [1764022800, '7.806'],
     [1764023100, '7.806'],
     [1764023400, '7.806'],
     [1764023700, '7.806'],
     [1764

* Average Memory usage over time

* Average Memory usage over time
* query_range
* `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="memory"})[1h:] offset 1h)`
* Average memory usage over the defined time range in the Wallaroo cluster.



In [40]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="memory"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {},
    'values': [[1764014400, '5619159313.066667'],
     [1764014700, '5633177309.866667'],
     [1764015000, '5647124155.733334'],
     [1764015300, '5660369328.355556'],
     [1764015600, '5673861569.422222'],
     [1764015900, '5688028029.155556'],
     [1764016200, '5701641528.888889'],
     [1764016500, '5714513419.377778'],
     [1764016800, '5722834750.577778'],
     [1764017100, '5720444307.911111'],
     [1764017400, '5715671443.911111'],
     [1764017700, '5713651740.444445'],
     [1764018000, '5713707684.9777775'],
     [1764018300, '5713827527.111111'],
     [1764018600, '5713719409.777778'],
     [1764018900, '5713914049.422222'],
     [1764019200, '5714019794.488889'],
     [1764019500, '5714122143.288889'],
     [1764019800, '5714111886.222222'],
     [1764020100, '5714751249.066667'],
     [1764020400, '5715206741.333333'],
     [1764020700, '5715392409.6'],
     [1764021000, '5715757192.

### Average Memory requests over time

* Average Memory requests over time
* query_range
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="memory"})[1h:] offset 1h)`
* Average memory requests over the defined time range in the Wallaroo cluster.


In [41]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="memory"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {},
    'values': [[1764014400, '18127781888'],
     [1764014700, '18127781888'],
     [1764015000, '18127781888'],
     [1764015300, '18127781888'],
     [1764015600, '18127781888'],
     [1764015900, '18127781888'],
     [1764016200, '18127781888'],
     [1764016500, '18127781888'],
     [1764016800, '18127781888'],
     [1764017100, '18127781888'],
     [1764017400, '18127781888'],
     [1764017700, '18127781888'],
     [1764018000, '18127781888'],
     [1764018300, '18127781888'],
     [1764018600, '18127781888'],
     [1764018900, '18127781888'],
     [1764019200, '18127781888'],
     [1764019500, '18127781888'],
     [1764019800, '18127781888'],
     [1764020100, '18127781888'],
     [1764020400, '18127781888'],
     [1764020700, '18127781888'],
     [1764021000, '18127781888'],
     [1764021300, '18127781888'],
     [1764021600, '18127781888'],
     [1764021900, '18127781888'],
     [1764022200, '181

### Average pipelines CPU usage over time

* Average pipelines CPU usage over time
* query_range
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_usage{resource="cpu"})[1h:] offset 1h)`
* Average CPU usage over the defined time range for an individual Wallaroo pipeline.


In [42]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_usage{resource="cpu"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'values': [[1764014400, '0.008022829029166668'],
     [1764014700, '0.007941153291666668'],
     [1764015000, '0.008027470070833334'],
     [1764015300, '0.007859062905555556'],
     [1764015600, '0.007630750926388889'],
     [1764015900, '0.007639744463888889'],
     [1764016200, '0.0076550476888888885'],
     [1764016500, '0.007607538625'],
     [1764016800, '0.007686739520833333'],
     [1764017100, '0.0077683091875'],
     [1764017400, '0.007837273087499999'],
     [1764017700, '0.007910831658333334'],
     [1764018000, '0.007927262533333333'],
     [1764018300, '0.008011655429166665'],
     [1764018600, '0.0079862143'],
     [1764018900, '0.008006672090277778'],
     [1764019200, '0.008205797156944444'],
     [1764019500, '0.008340867040277778'],
     [1764019800, '0.008454449686111112'],
     [1764020100, '0.0085505917'],
     [1764020400, '0.0086089636375'],

### Average pipelines CPU requested over time

* Average pipelines CPU requested over time
* query_range
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource="cpu"})[1h:] offset 1h)`
* Average number of CPUs requested over the defined time range for an individual Wallaroo pipeline.


In [43]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource="cpu"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'values': [[1764014400, '0.1'],
     [1764014700, '0.1'],
     [1764015000, '0.1'],
     [1764015300, '0.1'],
     [1764015600, '0.1'],
     [1764015900, '0.1'],
     [1764016200, '0.1'],
     [1764016500, '0.1'],
     [1764016800, '0.1'],
     [1764017100, '0.1'],
     [1764017400, '0.1'],
     [1764017700, '0.1'],
     [1764018000, '0.1'],
     [1764018300, '0.1'],
     [1764018600, '0.1'],
     [1764018900, '0.1'],
     [1764019200, '0.1'],
     [1764019500, '0.1'],
     [1764019800, '0.1'],
     [1764020100, '0.1'],
     [1764020400, '0.1'],
     [1764020700, '0.1'],
     [1764021000, '0.1'],
     [1764021300, '0.1'],
     [1764021600, '0.1'],
     [1764021900, '0.1'],
     [1764022200, '0.1'],
     [1764022500, '0.1'],
     [1764022800, '0.1'],
     [1764023100, '0.1'],
     [1764023400, '0.1'],
     [1764023700, '0.1'],
     [1764024000, '0.1'],
     [1764024

### Average pipelines GPU usage over time

* Average pipelines GPU usage over time
* query_range
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[1h:] offset 1h)`
* Average GPU usage over the defined time range for an individual Wallaroo pipeline.



In [44]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'values': [[1764058200, '1'],
     [1764058500, '1'],
     [1764058800, '1'],
     [1764059100, '1'],
     [1764059400, '1'],
     [1764059700, '1'],
     [1764060000, '1'],
     [1764060300, '1'],
     [1764060600, '1'],
     [1764060900, '1'],
     [1764061200, '1'],
     [1764061500, '1'],
     [1764061800, '1'],
     [1764062100, '1'],
     [1764062400, '1'],
     [1764062700, '1'],
     [1764063000, '1'],
     [1764063300, '1'],
     [1764063600, '1'],
     [1764063900, '1'],
     [1764064200, '1'],
     [1764064500, '1'],
     [1764064800, '1'],
     [1764065100, '1'],
     [1764065400, '1'],
     [1764065700, '1'],
     [1764066000, '1'],
     [1764066300, '1'],
     [1764066600, '1'],
     [1764066900, '1'],
     [1764067200, '1'],
     [1764067500, '1'],
     [1764067800, '1'],
     [1764068100, '1'],
     [1764068400, '1'],
     [1764068700, '1'],
     [1

### Average pipelines GPU requested over time

* Average pipelines GPU requested over time
* query_range
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)`
* Average number of GPUs requested over the defined time range for an individual Wallaroo pipeline. |


In [45]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'values': [[1764058200, '1'],
     [1764058500, '1'],
     [1764058800, '1'],
     [1764059100, '1'],
     [1764059400, '1'],
     [1764059700, '1'],
     [1764060000, '1'],
     [1764060300, '1'],
     [1764060600, '1'],
     [1764060900, '1'],
     [1764061200, '1'],
     [1764061500, '1'],
     [1764061800, '1'],
     [1764062100, '1'],
     [1764062400, '1'],
     [1764062700, '1'],
     [1764063000, '1'],
     [1764063300, '1'],
     [1764063600, '1'],
     [1764063900, '1'],
     [1764064200, '1'],
     [1764064500, '1'],
     [1764064800, '1'],
     [1764065100, '1'],
     [1764065400, '1'],
     [1764065700, '1'],
     [1764066000, '1'],
     [1764066300, '1'],
     [1764066600, '1'],
     [1764066900, '1'],
     [1764067200, '1'],
     [1764067500, '1'],
     [1764067800, '1'],
     [1764068100, '1'],
     [1764068400, '1'],
     [1764068700, '1'],
     [1

### Average pipelines Mem usage over time

* Average pipelines Mem usage over time
* query_range
* `avg_over_time(sum by(namespace) (wallaroo_kube_pod_resource_usage{resource="memory"})[1h:] offset 1h)`
* Average memory usage over the defined time range for an individual Wallaroo pipeline. |


In [46]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by(namespace) (wallaroo_kube_pod_resource_usage{resource="memory"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'values': [[1764014400, '16085777.066666666'],
     [1764014700, '16097723.733333332'],
     [1764015000, '16100795.733333332'],
     [1764015300, '16108919.466666667'],
     [1764015600, '16117145.6'],
     [1764015900, '16124040.533333333'],
     [1764016200, '16130935.466666667'],
     [1764016500, '16137824.711111112'],
     [1764016800, '16149139.91111111'],
     [1764017100, '16155983.644444445'],
     [1764017400, '16160864.711111112'],
     [1764017700, '16165711.644444445'],
     [1764018000, '16170490.311111111'],
     [1764018300, '16170763.377777778'],
     [1764018600, '16175610.311111111'],
     [1764018900, '16179774.577777777'],
     [1764019200, '16183170.844444444'],
     [1764019500, '16184297.244444445'],
     [1764019800, '16187966.577777777'],
     [1764020100, '16189747.2'],
     [1764020400, '16186060.8'],
     [1764020700, '16186743.4666666

### Average pipelines Mem requested over time

* Average pipelines Mem requested over time
* query_range
* `avg_over_time(sum by (namespace)(wallaroo_kube_pod_resource_requests{resource="memory"})[1h:] offset 1h)`
* Average amount of memory requested over the defined time range for an individual Wallaroo pipeline. |


In [47]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by (namespace)(wallaroo_kube_pod_resource_requests{resource="memory"})[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'values': [[1764014400, '134217728'],
     [1764014700, '134217728'],
     [1764015000, '134217728'],
     [1764015300, '134217728'],
     [1764015600, '134217728'],
     [1764015900, '134217728'],
     [1764016200, '134217728'],
     [1764016500, '134217728'],
     [1764016800, '134217728'],
     [1764017100, '134217728'],
     [1764017400, '134217728'],
     [1764017700, '134217728'],
     [1764018000, '134217728'],
     [1764018300, '134217728'],
     [1764018600, '134217728'],
     [1764018900, '134217728'],
     [1764019200, '134217728'],
     [1764019500, '134217728'],
     [1764019800, '134217728'],
     [1764020100, '134217728'],
     [1764020400, '134217728'],
     [1764020700, '134217728'],
     [1764021000, '134217728'],
     [1764021300, '134217728'],
     [1764021600, '134217728'],
     [1764021900, '134217728'],
     [1764022200, '134217728'],
     [1

### Pipeline inference log storage

* Pipeline inference log storage
* query_range
* `avg_over_time(sum by(topic) (topic_bytes)[1h:] offset 1h)`
* Average inference log storage used over the defined time range for an individual Wallaroo pipeline

In [48]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query_range"

# Retrieve the token 
headers = wl.auth.auth_header()

step = "5m" # the step of the calculation
# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
# Define the start and end times

data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# Convert to UTC and get the Unix timestamps

start_timestamp = int(data_start.astimezone(pytz.UTC).timestamp())
end_timestamp = int(data_end.astimezone(pytz.UTC).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = 'avg_over_time(sum by(topic) (topic_bytes)[1h:] offset 1h)'

#request parameters
params_rps = {
    'query': query,
    'start': start_timestamp,
    'end': end_timestamp,
    'step': step
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'matrix',
  'result': [{'metric': {'topic': 'workspace-121-pipeline-whisper-hf-byop-jcw-inference'},
    'values': [[1764014400, '0'],
     [1764014700, '0'],
     [1764015000, '0'],
     [1764015300, '0'],
     [1764015600, '0'],
     [1764015900, '0'],
     [1764016200, '0'],
     [1764016500, '0'],
     [1764016800, '0'],
     [1764017100, '0'],
     [1764017400, '0'],
     [1764017700, '0'],
     [1764018000, '0'],
     [1764018300, '0'],
     [1764018600, '0'],
     [1764018900, '0'],
     [1764019200, '0'],
     [1764019500, '0'],
     [1764019800, '0'],
     [1764020100, '0'],
     [1764020400, '0'],
     [1764020700, '0'],
     [1764021000, '0'],
     [1764021300, '0'],
     [1764021600, '0'],
     [1764021900, '0'],
     [1764022200, '0'],
     [1764022500, '0'],
     [1764022800, '0'],
     [1764023100, '0'],
     [1764023400, '0'],
     [1764023700, '0'],
     [1764024000, '0'],
     [1764024300, '0'],
     [1764024600, '0'],
   